# Binary Phase Field Benchmark

FiPy implementation of phase transormation in 2D with solutal driving force

**Do not edit `binary_phase_field.py`**. Generate the batch-runnable file from the notebook with
```bash
jupyter nbconvert binary_phase_field.ipynb --to python --output-dir=../scripts/
```

## Import Python modules

In [ ]:
import argparse
import json
import os
import re
import sys

Jupyter notebook handles some things differently than from the commandline

In [ ]:
isnotebook = False
try:
    from IPython import get_ipython
    isnotebook = (get_ipython().__class__.__name__ == "ZMQInteractiveShell")
except:
    pass

## Initialize
### Load parameters

In [ ]:
config = {
    "solvelog": snakemake.output[0], # file to for FiPy to log into
    "output": os.path.dirname(snakemake.output[0]), # directory to store results in
    "restart": None, # solution to initialize from
    "checkpoint_interval": 6., # frequency to save results
    "totaltime": 600., # duration of full simulation
    "numberOfElements": snakemake.params.config.get("size", 100000), # number of total cells in a Grid2D
    "solver": "pcg", # solver class to use
    "preconditioner": "none", # preconditioner class to use
    "sweeps": 5, # number of nonlinear sweeps to take
    "iterations": 1000, # maximum number of linear iterations to take for each sweep
    "tolerance": 1e-10, # linear solver tolerance
    "store_matrix": True, # store the matrix and RHS vector along with other output
    "gradient2thermal": 1, # strength of gradient energy relative to thermal energy
    "segregation2transformation": 1, # strength of segregation relative to transformation driving force
    "adsorption": 0, # strength of relative adsorption
    "profile": True, # store profiling statistics along with other output
    "view": True # whether to display results
}
config.update(snakemake.config)
config.update(snakemake.params.config)

if config["tolerance"] != "default":
    config["tolerance"] = float(config["tolerance"])

### Setup logging

In [ ]:
if "FIPY_LOG_CONFIG" not in os.environ:
    config["log_config"] = os.path.join(config["output"], "logger.json")

    with open(config["log_config"], "w") as f:
        f.write("""
{{
    "version": 1,
    "formatters": {{
        "default": {{
            "format": "%(asctime)s | %(levelname)s | %(name)s | %(funcName)s | %(message)s"
        }}
    }},
    "handlers": {{
        "serialfile": {{
            "class": "logging.FileHandler",
            "formatter": "default",
            "filename": "{solvelog}"
        }}
    }},
    "loggers": {{
        "fipy": {{
            "level": "DEBUG",
            "handlers": ["serialfile"]
        }}
    }}
}}
""".format(solvelog=config["solvelog"]))

    os.environ["FIPY_LOG_CONFIG"] = config["log_config"]

### Setup solver suite

In [ ]:
if "FIPY_SOLVERS" not in os.environ:
    os.environ["FIPY_SOLVERS"] = config["suite"]

### Import FiPy

In [ ]:
!conda info